In [1]:
"""
Created on Wed Apr 5 4:21:12 2017

@author: Batul Zamin
"""

'\nCreated on Wed Apr 5 4:21:12 2017\n\n@author: Batul Zamin\n'

In [2]:
# import statements
import pandas as pd
import numpy as np
from surprise.prediction_algorithms.knns import KNNWithMeans as knn
from surprise import Dataset
from surprise import evaluate

In [3]:
# reading the data
header1 = ['user_id', 'item_id', 'rating', 'timestamp']
df_data = pd.read_csv('ml-100k/u.data', sep='\t', names=header1)
header2 = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
df_user = pd.read_csv('ml-100k/u.user', sep='|', names=header2)
header3 = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure',
           'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
           'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
df_item = pd.read_csv('ml-100k/u.item', sep='|', names=header3, encoding='latin-1')

## KNN using the Surprise Package

In [4]:
data = Dataset.load_builtin('ml-100k')
data.split(n_folds=3)

In [5]:
algo = knn(k=10, min_k=1)

In [6]:
for trainset, testset in data.folds():
    algo.train(trainset)
    pred = algo.test(testset)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


In [7]:
perf = evaluate(algo, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm KNNWithMeans.

------------
Fold 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9807
MAE:  0.7730
------------
Fold 2
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9848
MAE:  0.7784
------------
Fold 3
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9782
MAE:  0.7704
------------
------------
Mean RMSE: 0.9812
Mean MAE : 0.7739
------------
------------


In [8]:
pred[:5]

[Prediction(uid='339', iid='190', r_ui=4.0, est=4.6569812469496625, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='655', iid='874', r_ui=4.0, est=2.0055058677249109, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='802', iid='565', r_ui=3.0, est=2.480293921488756, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='656', iid='689', r_ui=2.0, est=1.8816416225659389, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='381', iid='77', r_ui=2.0, est=3.6675021418886935, details={'actual_k': 10, 'was_impossible': False})]

## Predict the ratings for one user

In [150]:
# Enter user to recommend movies
u = input()

940


In [151]:
predict = []
for i in range(1682):
    predict.append(algo.predict(str(u), str(i), r_ui=float(df_data.rating[i])).est)

In [152]:
rec_movie = sorted(predict, reverse=True)

## Comparison of watched and recommended top 10 movies

In [153]:
watched_movie_index = list(df_data.item_id[df_data.user_id==int(u)])

In [154]:
watched_movie_title = []
for i in watched_movie_index:
    watched_movie_title.append(list(df_item.movie_title[df_item['movie_id'] == i+1]))

In [155]:
df1 = pd.DataFrame(columns=['watched'])
for i in watched_movie_title:
    df1.loc[-1] = i
    df1.index += 1
df1 = df1.reset_index(drop=True)

In [156]:
get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]

In [188]:
indexes = []
predict = list(predict)
count = 1
for i in rec_movie:
    if predict.index(i) in indexes:
        if count < len(get_indexes(i, predict)):
            indexes.append(get_indexes(i, predict)[count])
            count += 1
        else:
            count = 0
            indexes.append(get_indexes(i, predict)[count])
    else:
        indexes.append(predict.index(i))

In [190]:
rec_movie_index = [indexes[i] for i in range(len(indexes)) if indexes[i] not in watched_movie_index]

In [191]:
rec_movie_title = []
for i in rec_movie_index[:10]:
    rec_movie_title.append(list(df_item.movie_title[df_item['movie_id'] == i+1]))

In [192]:
df2 = pd.DataFrame(columns=['recommended'])
for i in rec_movie_title:
    df2.loc[-1] = i
    df2.index += 1
df2 = df2.reset_index(drop=True) 

In [193]:
df = pd.concat([df1, df2], ignore_index=True, axis=1)

In [195]:
df.head(10)

,0,1
0,"Sting, The (1973)",One Fine Day (1996)
1,Wolf (1994),"Cure, The (1995)"
2,Mr. Holland's Opus (1995),Carpool (1996)
3,Akira (1988),Cosi (1996)
4,Heat (1995),"Silence of the Palace, The (Saimt el Qusur) (1..."
5,M (1931),Prisoner of the Mountains (Kavkazsky Plennik) ...
6,As Good As It Gets (1997),Angel Baby (1995)
7,Ace Ventura: Pet Detective (1994),Rhyme & Reason (1997)
8,Career Girls (1997),Golden Earrings (1947)
9,Fierce Creatures (1997),Sprung (1997)
